In [2]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
df_interactions = pd.read_csv('../data/userid_trackid_timestamp.tsv', sep='\t')
print(df_interactions.shape)
df_interactions.head()

(252984396, 3)


,user_id,track_id,timestamp
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38


In [15]:
df_emma = pd.read_csv('../data/id_emma.tsv', sep='\t')
df_emma

,id,wond,tran,tend,nost,peac,joya,ener,sadn,tens
0,XEbyzfafxVpCCLK2,1.594081,-0.667002,-0.150606,-1.513499,-0.816678,0.428617,1.198807,-0.446335,0.612913
1,wpldgYOXl8u6XbLE,1.553820,2.420615,1.212955,-0.130002,0.431165,0.099829,1.764385,-0.196834,-1.268899
2,nltzxmV9aWW7F7z8,0.461760,-0.140861,-0.240865,-0.581115,-0.609828,1.255161,2.990274,-0.503040,-1.268899
3,IIfvFDMGH4mskejp,-0.641098,-0.064888,-0.945242,-1.094666,-0.689187,1.339656,0.696600,-0.628664,-0.671147
4,kXNEvIQ4JLPHMu3H,-0.432962,-0.497407,0.074756,-0.770869,0.790146,0.250230,-0.262315,-0.432901,-0.504749
...,...,...,...,...,...,...,...,...,...,...
504,ZR4UokUqcWBT4ERU,0.586800,2.189592,-0.253315,3.456587,-0.663044,1.923693,2.139593,-0.903227,-0.105909
505,zsDz17XVFUFYh4y7,0.857097,0.131091,2.164516,1.233984,2.145191,-0.777977,-1.442610,-0.055070,-1.054949
506,ZSnDQ7mhvjJLx9IG,-1.160694,-1.318940,-0.307150,-0.417539,-0.161656,-0.601481,-0.900951,-0.395868,-0.305295
507,ZVjUg4rlqeq2zk7C,-0.127027,-0.082542,-0.214522,-0.492390,0.360165,-0.382865,-0.398755,-0.390467,-0.751960


In [16]:
df_interactions = df_interactions[df_interactions['track_id'].isin(df_emma.id)]
print(df_interactions.shape)
df_interactions.head()

(3600728, 3)


,user_id,track_id,timestamp
1648,52740,rFJl0J6qKPImZWOQ,2013-03-03 19:41:52
1718,52740,rFJl0J6qKPImZWOQ,2013-02-03 15:55:45
2141,52740,rFJl0J6qKPImZWOQ,2013-02-01 21:45:34
2148,52740,rFJl0J6qKPImZWOQ,2013-01-31 21:59:41
2155,52740,rFJl0J6qKPImZWOQ,2013-01-31 19:15:30


In [18]:
def get_periods_with_min_plays(df_interactions, min_plays=5, freq='M'):
    df_interactions['period'] = pd.to_datetime(df_interactions['timestamp']).dt.to_period(freq)
    
    period_track_plays = df_interactions.groupby(['period', 'track_id']).size().reset_index(name='plays')

    period_track_plays = period_track_plays[period_track_plays['plays'] >= min_plays]
    
    period_summary = period_track_plays.groupby('period').agg(
        min_plays=('plays', 'min'),
        tracks_count=('track_id', 'count'),
    )

    return period_summary.sort_values('tracks_count', ascending=False)

# Get qualifying periods
freq = 'M'  # Change this to 'Y' for years, 'W' for weeks, 'D' for days, etc.
min_plays = 5
qualifying_periods = get_periods_with_min_plays(df_interactions, min_plays=min_plays, freq=freq)
qualifying_periods

,min_plays,tracks_count
period,,
2014-05,5,196
2012-05,5,195
2014-07,5,194
2014-01,5,194
2014-04,5,193
...,...,...
2005-07,5,7
2005-06,5,6
2005-05,5,6
